# Data preparation - Proyecto integrador

El análisis de texto es un campo creciente. Las ingentes cantidades de texto, representan una oportunidad para las organizaciones que desean encontrar patrones, conocer las opiniones de sus consumidores, analizar posibles coocurrencias, e inclusive, generar predicciones sobre las palabras que el usuario digitará, o traducir en tiempo real un texto entre distintos idiomas.

En el notebook se busca utilizar las funciones disponibles para obtener información líquida suceptible de analizar basados en el cálculo de frecuencua de palabras, aplicación de term frencuency e idf y posteriormente el Bag Of Words para la base de datos que fue extraida de Twitter para la cuenta de avianca.

Diccionario a utilizar: download es_core_news_sm
Para más información sobre spacy: https://spacy.io/

Los estudiantes que participaron en el desarrollo de este proyecto son:
    
    1. Jorge Luis Renteria
    2. Edgar Leandro Jimenez
    3. Jesus Alberto Arcia
    
Universidad Eafit, 2019.

### Importacion de librerias necesarias

In [5]:
import codecs
import spacy
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP
import pandas as pd
import numpy as np
import random
import re
import string
import nltk
from nltk.collocations import *
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import heapq
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

#nltk.download('stopwords')
nlp=spacy.load('es_core_news_sm')


### Carga de datos

In [2]:
df = pd.read_excel('TweetsEntrenamiento.xlsx')

# Extraemos el texto
corpus= list(df['Texto'])
MiListaStops=list(set(stopwords.words('spanish')))

### Limpieza de todos los tweets

In [3]:
for i,tweet in enumerate(corpus):
    tweet=re.sub('@([\w.]+ )','',re.sub('https.*','',tweet))
    tweet=re.sub('#[\w]*','',tweet)
    tweet = tweet.lower()
    tweet = re.sub(r'\W',' ',re.sub(r'\s+',' ',tweet))
    tweet = re.sub('¡','',re.sub('°','',tweet))
    tweet=tweet.translate(str.maketrans('', '', string.punctuation))
    for word in tweet.split():
        if str(word.lower()) in MiListaStops:
            tweet=re.sub(r'\b'+str(word)+r'\b','',tweet)
    tweet=re.sub(' +',' ',tweet)
    corpus[i] = tweet

## Realizamos el conteo de las palabras 

### Tokens: 

Una base de datos de texto (o corpus) es una agrupación de bytes. El texto en su forma más pura, es una colección de bytes (o caracteres). La mayoria de veces es útil agrupar estos caracteres en unidades continuas llamadas tokens. En español, al igual que en la mayoria de los idiomas occidentales, un token corresponde a palabras y sequencias numericas separadas por espacios en blanco o signos de puntuación. El proceso de reducir un texto a tokens se conoce como tokenización.


### Lemmas
Lemmas: Lemas corresponde a la raíz de una palabra. Considere por ejemplo la palabra correr. Esta puede tener distintas formas como corriendo, corrí, correré, entre otras. En este caso, resulta útil reducir la palabra a su raíz o lemma. En este caso, este proceso se conoce como lematization. Obtener los lemas es bastante sencillo una vez se ha procesado el texto con la funcion nlp de Spacy, y se logra a través de la función .lemma_

In [6]:
wordfreq = {}
for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    tokens=nlp(sentence)
    for token in tokens:
        if not str(token) in list([' ']):    
            token=token.lemma_
            if token not in wordfreq.keys():
                wordfreq[token] = 1
            else:
                wordfreq[token] += 1

In [7]:
# Exraemos las palabras mas frecuentes (Para este caso utilizamos todas)
most_freq = heapq.nlargest(4786, wordfreq, key=wordfreq.get)

### Creamos una lista con todos los tweets y su tokenizacion

In [8]:
milista=[]
for document in corpus:
    milista.append(list(str(nlp(document)).split()))

### Proceso para calcular los tf-idf

In [9]:
word_idf_values = {}

for token in most_freq:
    doc_containing_word = 0
    for document in milista:
        if str(token) in document:
            doc_containing_word += 1
    word_idf_values[token] = np.log(len(corpus)/(1 + doc_containing_word))

In [10]:
word_tf_values = {}

for token in most_freq:
    sent_tf_vector = []
    for document in milista:
        doc_freq = document.count(token)
        if len(document)==0:
            continue
        word_tf = doc_freq/len(document)
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector

In [11]:
tfidf_values = []

for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append(tfidf_sentences)

In [12]:
tf_idf_model = np.asarray(tfidf_values)
tf_idf_model = np.transpose(tf_idf_model)    

In [13]:
tf_idf_model

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.42967741, 0.42967741,
        0.42967741]])

In [15]:
type(tf_idf_model)

numpy.ndarray

### Convertimos a data frame el resultado final

In [16]:
df_final = pd.DataFrame(tf_idf_model)

### Ahora asociamos el sentimiento a cada fila (Tweet)

In [18]:
df_final['label']=df['Calificación sentimiento']

### Exportamos a csv para guardar en nube

In [20]:
df_final.to_csv("bow" + ".csv", encoding='utf')